# 09 — Run All Experiments (E1–E5)

Este notebook ejecuta el paquete completo de experimentos con todos los modelos y exógenas.

**Supuesto experimental (oracle exog):** todas las exógenas están disponibles durante el horizonte de predicción.


In [1]:
# Imports y configuración base
from __future__ import annotations

import json
import sys
from pathlib import Path

import numpy as np
import pandas as pd

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.common import TEST_WEEKS, load_data, make_features, validate_split_consistency
from src.experiments import (
    run_E1_walk_forward,
    run_E2_last_39_weeks,
    run_E3_loso,
    run_E4_train35_test10low,
    run_E5_unemployment_shock,
    collect_summary_metrics,
 )

from src.models import (
    SarimaxForecaster,
    ProphetForecaster,
    DeepARForecaster,
    LSTMForecaster,
    TransformerForecaster,
 )

SEED = 42
np.random.seed(SEED)

DATA_PATH = PROJECT_ROOT / 'data' / 'Walmart_Sales.csv'
OUTPUTS_DIR = PROJECT_ROOT / 'outputs'
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

print('Project root:', PROJECT_ROOT)
print('Data path:', DATA_PATH)

Project root: /home/sagemaker-user/TFMAXEL
Data path: /home/sagemaker-user/TFMAXEL/data/Walmart_Sales.csv


## Configuración común
- Lags y rollings (solo pasado)
- Exógenas obligatorias
- Horizonte de test = últimas 39 semanas

In [2]:
config = {
    'seed': SEED,
    'lags': [1, 2, 4, 8, 52],
    'rollings': [4, 8, 12],
    'exog_cols': ['Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment'],
    'add_calendar': True,
    'lookback': 52,
    'test_weeks': TEST_WEEKS,
    'output_dir': str(OUTPUTS_DIR),
    'epochs': 20,
    'batch_size': 64,
    'lr': 1e-3,
}

df = load_data(DATA_PATH)
df_feat, feature_cols = make_features(
    df,
    lags=config['lags'],
    rollings=config['rollings'],
    add_calendar=config['add_calendar'],
    group_col='Store',
)
config['feature_cols'] = feature_cols

# Nota: los modelos que usan lags/rollings calcularán features de forma recursiva en predicción
df_feat.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,lag_1,lag_2,...,lag_52,roll_mean_4,roll_std_4,roll_mean_8,roll_std_8,roll_mean_12,roll_std_12,weekofyear,month,year
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2,2010
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,1643690.90,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,2,2010
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,1641957.44,1643690.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2,2010
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,1611968.17,1641957.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,2,2010
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,1409727.59,1611968.17,...,NaN,1576836.025,112353.415114,NaN,NaN,NaN,NaN,9,3,2010


## Metadata de experimentos
Se guarda en `outputs/metadata_experiments.json` con fechas y versiones.

In [3]:
try:
    from importlib.metadata import version
except Exception:
    version = None

def safe_version(pkg: str) -> str | None:
    if version is None:
        return None
    try:
        return version(pkg)
    except Exception:
        return None

metadata_exp = {
    'seed': SEED,
    'exog_cols': config['exog_cols'],
    'lags': config['lags'],
    'rollings': config['rollings'],
    'lookback': config['lookback'],
    'test_weeks': config['test_weeks'],
    'libs': {
        'numpy': safe_version('numpy'),
        'pandas': safe_version('pandas'),
        'statsmodels': safe_version('statsmodels'),
        'prophet': safe_version('prophet'),
        'gluonts': safe_version('gluonts'),
        'torch': safe_version('torch'),
    },
    'oracle_exog_assumption': 'All exogenous covariates are assumed available over the forecast horizon (oracle scenario).',
}

meta_path = OUTPUTS_DIR / 'metadata_experiments.json'
meta_path.write_text(json.dumps(metadata_exp, indent=2, ensure_ascii=False), encoding='utf-8')
print('Wrote:', meta_path)

Wrote: /home/sagemaker-user/TFMAXEL/outputs/metadata_experiments.json


## Instancia de modelos
Si falta una dependencia (Prophet/GluonTS), el modelo se omite con un aviso.

In [4]:
models = []

# M1 SARIMAX
models.append(SarimaxForecaster())

# M2 Prophet
try:
    models.append(ProphetForecaster())
except Exception as exc:
    print('Skipping Prophet:', exc)

# M3 LSTM (mover antes que DeepAR)
try:
    models.append(LSTMForecaster())
except Exception as exc:
    print('Skipping LSTM:', exc)

# M4 DeepAR
try:
    models.append(DeepARForecaster())
except Exception as exc:
    print('Skipping DeepAR:', exc)

# M5 Transformer
try:
    models.append(TransformerForecaster())
except Exception as exc:
    print('Skipping Transformer:', exc)

models

[SarimaxForecaster(name='sarimax_exog', is_global=False),
 ProphetForecaster(name='prophet_regressors', is_global=False),
 LSTMForecaster(name='lstm_exog', is_global=True),
 DeepARForecaster(name='deepar_exog', is_global=True),
 TransformerForecaster(name='transformer_exog', is_global=True)]

## Ejecución por modelo (para reintentar)
Usa estas utilidades para correr cada modelo por separado y evitar relanzar todo si uno falla.

In [5]:
def get_model_by_name(name: str):
    for m in models:
        if m.name == name:
            return m
    raise ValueError(f"Model {name} not found")

def run_model_suite(model):
    print(f"== Running {model.name} ==")
    # E1 + E2
    try:
        run_E1_walk_forward([model], df, config)
        split_info_local = run_E2_last_39_weeks([model], df, config)
    except Exception as exc:
        print(f"[{model.name}] stopped in E1/E2: {exc}")
        return

    # E3 + E4 (solo globales)
    if getattr(model, "is_global", False):
        try:
            run_E3_loso([model], df, config)
            run_E4_train35_test10low([model], df, config)
        except Exception as exc:
            print(f"[{model.name}] stopped in E3/E4: {exc}")

    # E5
    try:
        run_E5_unemployment_shock([model], df, config, shock_pct=0.15)
    except Exception as exc:
        print(f"[{model.name}] stopped in E5: {exc}")

    return split_info_local

### Ejecutar todos los modelos con tolerancia a fallos
Esta celda recorre todos los modelos y sigue con el siguiente si alguno falla.

In [6]:
for m in models:
    run_model_suite(m)

== Running sarimax_exog ==


== Running prophet_regressors ==


== Running lstm_exog ==


[LSTM] Reducing lookback from 52 to 39 due to limited per-store history (min len=40 after dropna).


== Running deepar_exog ==


...

INFO: Epoch 5, global step 300: 'train_loss' reached 9.98051 (best 9.98051), saving model to '/home/sagemaker-user/TFMAXEL/notebooks/lightning_logs/version_72/checkpoints/epoch=5-step=300.ckpt' as top 1


INFO: Epoch 6, global step 350: 'train_loss' reached 9.96899 (best 9.96899), saving model to '/home/sagemaker-user/TFMAXEL/notebooks/lightning_logs/version_72/checkpoints/epoch=6-step=350.ckpt' as top 1


INFO: Epoch 7, global step 400: 'train_loss' reached 9.94704 (best 9.94704), saving model to '/home/sagemaker-user/TFMAXEL/notebooks/lightning_logs/version_72/checkpoints/epoch=7-step=400.ckpt' as top 1


INFO: Epoch 8, global step 450: 'train_loss' reached 9.86810 (best 9.86810), saving model to '/home/sagemaker-user/TFMAXEL/notebooks/lightning_logs/version_72/checkpoints/epoch=8-step=450.ckpt' as top 1


INFO: Epoch 9, global step 500: 'train_loss' was not in top 1


### Ejecutar un modelo específico
Ejemplos de uso; cambia el nombre si quieres otro modelo.

In [ ]:
# Ejecutar solo LSTM
run_model_suite(get_model_by_name("lstm_exog"))

# Ejecutar solo Transformer
run_model_suite(get_model_by_name("transformer_exog"))

## Ejecutar E1 y E2 (todos los modelos)

In [ ]:
run_E1_walk_forward(models, df, config)
split_info = run_E2_last_39_weeks(models, df, config)
split_info

## Ejecutar E3 y E4 (solo modelos globales)

In [ ]:
models_global = [m for m in models if getattr(m, 'is_global', False)]
run_E3_loso(models_global, df, config)
run_E4_train35_test10low(models_global, df, config)

## Ejecutar E5 (shock de desempleo +15%)

In [ ]:
run_E5_unemployment_shock(models, df, config, shock_pct=0.15)

## Resumen de métricas

In [ ]:
# Resumen de métricas (una sola escritura)
from src.experiments import collect_summary_metrics

summary = collect_summary_metrics(OUTPUTS_DIR)
summary_path = OUTPUTS_DIR / "experiments" / "summary_metrics.csv"
summary_path.parent.mkdir(parents=True, exist_ok=True)
summary.to_csv(summary_path, index=False)
summary_path

In [ ]:
# Validación automática del split (test_weeks=39)
validation = validate_split_consistency(OUTPUTS_DIR, test_weeks=TEST_WEEKS)
print(validation)

# En una corrida "from scratch", es normal que no existan predicciones todavía.
issues = list(validation.get("issues", []) or [])
only_missing_preds = (issues == ["No test_predictions.csv files found under outputs_dir"])
if not validation.get("ok", False) and not only_missing_preds:
    raise AssertionError(issues)
elif only_missing_preds:
    print("[WARN] Split OK, pero todavía no hay test_predictions.csv (aún no se ejecutaron modelos).")
